## Preprocess Data to perform Liner Regression
---

In [7]:
import pandas as pd

In [8]:
olympics_data = pd.read_csv('olympics_data.csv')
print(olympics_data.shape)
olympics_data.head()

(271116, 16)


,sno,ID,Name,Sex,Age,Height,Weight,Country,NOC,Games,Year,Season,City,Sport,Event,Medal
0,0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


### Summer Olympics Data
---

In [9]:
# Filtering the dataFrame for 'Summer' season
summer_df = olympics_data[olympics_data['Season'] == 'Summer']

print(summer_df.shape)
summer_df.head()

(222552, 16)


,sno,ID,Name,Sex,Age,Height,Weight,Country,NOC,Games,Year,Season,City,Sport,Event,Medal
0,0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
26,26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN


### Host Country Data
---

In [10]:
host_df = pd.read_csv('summer_host_country.csv')
host_df

FileNotFoundError: [Errno 2] No such file or directory: 'summer_host_country.csv'

### Merge summer_df and host_df
---

In [ ]:
summer_host = pd.merge(summer_df, host_df, how = 'left', on = 'Year')
print(summer_host.shape)
summer_host.head()

### Count of Athletes, Sports and Events
---

In [ ]:
athlete_df = summer_host[['Year','Host_Country','Country','Name','Sport','Event']]

print(athlete_df.shape)
athlete_df.head()

In [ ]:
athlete_df = athlete_df.groupby(['Year','Host_Country','Country'])[['Name','Sport','Event']]\
                        .nunique()\
                        .reset_index()

print(athlete_df.shape)
athlete_df.head(10)

In [ ]:
athlete_df = athlete_df.rename(columns={'Name':'Athletes', 'Sport':'Sports', 'Event':'Events'})\
                        .sort_values(['Year','Athletes'],ascending = [True,False])\
                        .reset_index(drop = True)
print(athlete_df.shape)
athlete_df.head(10)

### Medal Count
---

In [ ]:
# Selecting only required columns
medals_df = summer_host[['Year','Host_Country','Country','Event','Medal']]

# Drop duplicate rows for team events.
medals_df = medals_df.drop_duplicates(['Country','Year','Event','Medal'])

# Renaming 'Medal column' to 'Medals'
medals_df = medals_df.rename(columns={'Medal': 'Medals'})\
                        .reset_index(drop = True)

print(medals_df.shape)
medals_df.head()

In [ ]:
medals_df = medals_df[['Year','Host_Country','Country','Medals']]\
                .sort_values(['Year','Country'], ascending = [True,True])\
                .reset_index(drop = True)

print(medals_df.shape)
medals_df.head()

In [ ]:
medals_df['Gold'] = [1 if cell == 'Gold' else 0 for cell in medals_df['Medals']]
medals_df['Silver'] = [1 if cell == 'Silver' else 0 for cell in medals_df['Medals']]
medals_df['Bronze'] = [1 if cell == 'Bronze' else 0 for cell in medals_df['Medals']]

print(medals_df.shape)
medals_df.head()

In [ ]:
medals_df = medals_df.drop(['Medals'], axis=1)
print(medals_df.shape)
medals_df.head()

In [ ]:
# Find total count of medals per year per country
medals_df = medals_df.groupby(['Year','Host_Country','Country'], sort = False).sum().reset_index()
print(medals_df.shape)
medals_df.head()

In [ ]:
medals_df['Medals'] = medals_df['Gold'] + medals_df['Silver'] + medals_df['Bronze']

print(medals_df.shape)
medals_df.head()

In [ ]:
medals_df = medals_df.sort_values(['Year','Medals','Country'], ascending = [True,False,True])

medals_df = medals_df.reset_index(drop=True)

print(medals_df.shape)
medals_df.head()

### Merging Athlete_count & Medals_count
---

In [ ]:
final_df = athlete_df.merge(medals_df, how ='left').fillna(0)
final_df = final_df.sort_values(['Year','Medals','Country'], ascending = [True,False,True])

print(final_df.shape)
final_df.head()

In [ ]:
final_df['Medals'] = final_df['Medals'].astype(int)

final_df['Gold'] = final_df['Gold'].astype(int)

final_df['Silver'] = final_df['Silver'].astype(int)

final_df['Bronze'] = final_df['Bronze'].astype(int)

print(final_df.shape)
final_df.head(10)

In [ ]:
# Adding 'Host column'
final_df['Host'] = 0
final_df.loc[final_df['Country'] == final_df['Host_Country'], 'Host'] = 1

print(final_df.shape)
final_df.head()

In [ ]:
# Dropping 'Host_country' column and rearranging other columns
final_df = final_df[['Year', 'Country', 'Host', 'Athletes', 'Sports', 'Events', 'Gold', 'Silver', 'Bronze', 'Medals']]

print(final_df.shape)
final_df.head()

In [ ]:
final_df.to_csv('summer_athlete_medals_count.csv',index = False)

## Top 25 countries
---

In [ ]:
top_25 = final_df.drop(['Year'], axis = 1)
top_25.head()

In [ ]:
top_25 = top_25.groupby(['Country']).sum().sort_values(['Medals'], ascending = False).reset_index()
top_25

In [ ]:
top_25 = top_25.iloc[0:25 ,:]
top_25

In [ ]:
top_25.to_csv('top_25.csv',index = False)

In [ ]:
print(len(summer_df['Country'].unique()))
#print(summer_df['Country'].unique())

print(len(athlete_df['Country'].unique()))
#print(athlete_df['Country'].unique())

print(len(final_df['Country'].unique()))
#print(final_df['Country'].unique())